### imports and globals

In [18]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
import pandas as pd

In [22]:
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI

from enum import Enum, auto
from strenum import LowercaseStrEnum

from langchain.prompts import PromptTemplate
from langchain.output_parsers import PandasDataFrameOutputParser

#from langchain_google_genai.chat_models import ChatGoogleGenerativeAI


In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
OPEN_AI_KEY = os.environ["OPEN_AI_KEY"]

### prepare promt template


In [23]:
# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    columns = ['Meal ID',
               'Date',
               'Time',
               'Meal component',
               'Quantity per serving',
               'Carbohydrates',
               'Protein',
               'Fat',
               'Calories'],
    data = [['1',
            '2025-02-25',
            '12:00',
            'Brown Bread (2 slices)',
            '2 slices',
            '20',
            '5',
            '5',
            '200']]
)

parser = PandasDataFrameOutputParser(dataframe=df)

In [75]:
nutri_prompt_template = ("You are a search assistant. The human user will give you a description of their meals, portion size and the time they consumed it. Your task is to analyse the meals, log the date and time of each meal,"
    "and then decompose the meal and its ingredients into their nutrient content. Use google search tool to get nutrient data from https://www.fatsecret.com/calories-nutrition/ if possible. MAke sensible guesses about portions where user does not provide"
    "Output the amount of carbohydrates, fats, proteins in grams, and calories. Also highlight any other key nutrients such as vitamins and minerals present in that portion"
    "Please give a unique id to each meal, that is repeated across its components. Answer with the nutritional breakdown of the in a tabular format (and nothing else) with columns for: "
    "- Meal ID"
    "- Date"
    "- Time"
    "- Meal component"
    "- Quantity per serving"
    "- Carbohydrates"
    "- Protein"
    "- Fat"
    "- Calories"
    "Only answer with the table, and nothing else.")

In [76]:
# Set up the prompt.
prompt_template = PromptTemplate(
    template = nutri_prompt_template + "\n{format_instructions}\n{meal_description}\n",
    input_variables=["meal_description"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [77]:
meal_description = ("Today is 2025-02-25. At noon I ate 1 sandwich made from 2 slices brown bread, tomato," 
              "1 slice of cheese and cucumber. At dinner 9pm, I ate 2 rotis with 1 bowl (100g) of chicken curry and 1 bowl of raita.")


# Generate the prompt
filled_prompt = prompt_template.invoke({"meal_description": meal_description})


### access GenAI chat API

In [78]:
class MODEL_ID(LowercaseStrEnum):
    GEMINI_1_5_FLASH_8B = "gemini-1.5-flash-8b"
    GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [79]:

# Define custom GenerateContentConfig parameters
client_options = {
    "temperature": 0.7,  # Controls randomness
    "max_output_tokens": 100,  # Limit on output tokens
    "stop_sequences": ["\n"],   # Optional stop sequences
}


# Initialize ChatGoogleGenerativeAI with client options
llm = ChatGoogleGenerativeAI(
    model = MODEL_ID.GEMINI_2_0_FLASH,
    google_api_key = GOOGLE_API_KEY,
    max_output_tokens = 500,
    temperature=0,
        top_p=0.95,
        top_k=20,
  config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    ))
   #client_options =  {'generate_content_config': {'max_output_tokens': 100}})

# Invoke the model with a prompt
response = llm.invoke(filled_prompt.text )
print(response.content)


```
Meal ID: Date: Time: Meal component: Quantity per serving: Carbohydrates: Protein: Fat: Calories
1:2025-02-25:12:00:Brown Bread:2 slices:24:4:2:130
1:2025-02-25:12:00:Tomato:50g:2:1:0:10
1:2025-02-25:12:00:Cheese:1 slice:1:7:9:115
1:2025-02-25:12:00:Cucumber:50g:2:0:0:8
2:2025-02-25:21:00:Roti:2:62:8:3:310
2:2025-02-25:21:00:Chicken Curry:100g:8:25:10:220
2:2025-02-25:21:00:Raita:1 bowl:10:3:5:90
```


In [61]:
REPLACEMENTS = ["```\n", "\n```", "```"]
response_text = response.text()
for replacement in REPLACEMENTS:
    response_text = response_text.replace(replacement, "")

In [63]:
#lines = response_text.split('\n')
#parsed_data = [line.split('|') for line in lines[:-1]]
parser.parse(response_text)

OutputParserException: Request 'Meal ID,Date,Time,Meal component,Quantity per serving,Carbohydrates,Protein,Fat,Calories
1,2025-02-25,12:00,Brown Bread,2 slices,24,4,2,130
1,2025-02-25,12:00,Tomato,50g,2,0.5,0.1,10
1,2025-02-25,12:00,Cheese,25g,1,5,7,90
1,2025-02-25,12:00,Cucumber,50g,2,0.5,0.1,8
2,2025-02-25,21:00,Roti,2,60,8,2,320
2,2025-02-25,21:00,Chicken Curry,100g,8,25,15,275
2,2025-02-25,21:00,Raita,1 bowl,10,2,5,90' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [64]:
from io import StringIO

csv_data = StringIO(response_text)
pd.read_csv(csv_data)

,Meal ID,Date,Time,Meal component,Quantity per serving,Carbohydrates,Protein,Fat,Calories
0,1,2025-02-25,12:00,Brown Bread,2 slices,24,4.0,2.0,130
1,1,2025-02-25,12:00,Tomato,50g,2,0.5,0.1,10
2,1,2025-02-25,12:00,Cheese,25g,1,5.0,7.0,90
3,1,2025-02-25,12:00,Cucumber,50g,2,0.5,0.1,8
4,2,2025-02-25,21:00,Roti,2,60,8.0,2.0,320
5,2,2025-02-25,21:00,Chicken Curry,100g,8,25.0,15.0,275
6,2,2025-02-25,21:00,Raita,1 bowl,10,2.0,5.0,90


### try google gemini directly

In [80]:
client = genai.Client(api_key=GOOGLE_API_KEY)

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response_direct = client.models.generate_content(
    model=MODEL_ID.GEMINI_2_0_FLASH,
    contents=filled_prompt, 
    config=types.GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)


In [89]:
print(response_direct_text)

| Meal ID | Date       | Time  | Meal component | Quantity per serving | Carbohydrates (g) | Protein (g) | Fat (g) | Calories |
| ------- | ---------- | ----- | --------------- | -------------------- | ----------------- | ----------- | ------- | -------- |
| 1       | 2025-02-25 | 12:00 | Brown Bread    | 2 slices             | 28                | 8           | 2       | 160      |
| 1       | 2025-02-25 | 12:00 | Tomato         | 50g                | 2                 | 0.5         | 0.1     | 10       |
| 1       | 2025-02-25 | 12:00 | Cheese         | 1 slice              | 1                 | 6           | 8       | 100      |
| 1       | 2025-02-25 | 12:00 | Cucumber       | 50g                | 2                 | 0.4         | 0.1     | 8        |
| 2       | 2025-02-25 | 21:00 | Roti           | 2                    | 60                | 8           | 2       | 320      |
| 2       | 2025-02-25 | 21:00 | Chicken Curry  | 100g               | 8                 | 25          | 15

In [87]:
REPLACEMENTS = ["```\n", "\n```", "```", "Analysis:\n\n"]
response_direct_text = response_direct.text
for replacement in REPLACEMENTS:
    response_direct_text = response_direct_text.replace(replacement, "")

In [88]:
from io import StringIO

csv_data = StringIO(response_direct_text)
pd.read_csv(csv_data)

,| Meal ID | Date | Time | Meal component | Quantity per serving | Carbohydrates (g) | Protein (g) | Fat (g) | Calories |
0,| ------- | ---------- | ----- | -------------...
1,| 1 | 2025-02-25 | 12:00 | Brown Bread ...
2,| 1 | 2025-02-25 | 12:00 | Tomato ...
3,| 1 | 2025-02-25 | 12:00 | Cheese ...
4,| 1 | 2025-02-25 | 12:00 | Cucumber ...
5,| 2 | 2025-02-25 | 21:00 | Roti ...
6,| 2 | 2025-02-25 | 21:00 | Chicken Curry...
7,| 2 | 2025-02-25 | 21:00 | Raita ...


## agent

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Combine LLM and tools into an agent
tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Example query that requires search
query = "What are the latest advancements in AI research? Please answer in not more than 5 lines"

response = agent.run(query)

print(response)


### try formatting into a dataframe

In [113]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = MODEL_ID.GEMINI_1_5_FLASH_8B,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [160]:
# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    columns = ['Meal ID',
               'Date',
               'Time',
               'Meal component',
               'Quantity per serving',
               'Carbohydrates',
               'Protein',
               'Fat',
               'Calories'],
    data = [['1',
            '2025-02-25',
            '12:00',
            'Brown Bread (2 slices)',
            '2 slices',
            '20',
            '5',
            '5',
            '200']]
)

# Set up a parser + inject instructions into the prompt template.
from langchain.output_parsers import PandasDataFrameOutputParser

parser = PandasDataFrameOutputParser(dataframe=df)

### openai trial

In [169]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
model = ChatOpenAI(temperature=0)